# Computation in python

In [7]:
x = 2
y = 3
z = (x * y) + (x * x) + 3
z

13

# Computational Graph in tensorflow

In [3]:
import tensorflow as tf

In [30]:
tf.reset_default_graph()

with tf.name_scope("scope1"):
    x = tf.Variable(3, name="x")
    y = tf.Variable(2, name="y")
    z = (x * y) + (x * x) + 3
z

<tf.Tensor 'scope1/add_1:0' shape=() dtype=int32>

In [31]:
tf.global_variables()
# tf.get_default_graph().get_operations()
# tf.get_collection(key=tf.GraphKeys.GLOBAL_VARIABLES, scope='scope1')


[<tf.Variable 'scope1/x:0' shape=() dtype=int32_ref>,
 <tf.Variable 'scope1/y:0' shape=() dtype=int32_ref>]

In [29]:
from IPython.display import clear_output, Image, display, HTML
import numpy as np

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
    
show_graph(tf.get_default_graph())

In [6]:
tf.reset_default_graph()
G1 = tf.Graph()
with G1.as_default():
    x = tf.Variable(3, name="x")
    y = tf.Variable(2, name="y")
    z = x * y
    
G2 = tf.Graph()
with G2.as_default():
    x = tf.Variable(3, name="x")
    y = tf.Variable(2, name="y")
    z = x + y
    

In [7]:
show_graph(G2)

# Tensorflow Session

In [8]:
tf.reset_default_graph()
x = tf.Variable(3, name="x")
y = tf.constant(2, name="y")
z = (x * y) + (x * x) + 3

In [9]:
session = tf.Session()
session.run(x.initializer)
print(session.run(z))
session.close()

18


In [10]:
with tf.Session() as ses:
    x.initializer.run()
    print(ses.run(z))

18


In [11]:
tf.reset_default_graph()
x = tf.Variable(3.0, name="x")
y = tf.constant(2.0, name="y")
e = tf.random_normal(shape=[], mean=0, stddev=1.0)
z = x + y + e
w = x + y + e
init_operation = tf.global_variables_initializer()
show_graph(tf.get_default_graph())

In [12]:
with tf.Session() as ses:
    ses.run(init_operation)
    print(ses.run(z))
    print(ses.run(w))
    print(ses.run([z, w]))
    

5.8576393
5.630676
[5.1873255, 5.1873255]


In [13]:
tf.reset_default_graph()
tf.set_random_seed(42)
x = tf.Variable(3.0, name="x")
y = tf.constant(2.0, name="y")
e = tf.random_normal(shape=[], mean=0, stddev=1.0)
z = x + y + e
w = x + y + e
init_operation = tf.global_variables_initializer()
with tf.Session() as ses:
    ses.run(init_operation)
    print(ses.run(z))
    print(ses.run(w))
    print(ses.run([z, w]))

5.1915555
4.2228513
[5.7696757, 5.7696757]


# Linear Regression with Tensorflow

In [14]:
import numpy as np
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
m, n = housing.data.shape
print("Shape of X", housing.data.shape)
print("Shape of Y", housing.target.shape)
X_data = housing.data
Y_data = np.reshape(housing.target, [-1, 1])

Shape of X (20640, 8)
Shape of Y (20640,)


## Analytical Solution

In [15]:
tf.reset_default_graph()
X = tf.constant(X_data, dtype=tf.float32, name='X')
Y = tf.constant(Y_data, dtype=tf.float32, name='Y')
X_transpose = tf.transpose(X, name='X_transpose')
A = tf.matmul(X_transpose, X, name='A')
B = tf.linalg.inv(A, name='B')
C = tf.matmul(B, X_transpose, name='C')
W = tf.matmul(C, Y, name='W')

with tf.Session() as ses:
    w_evaluated = ses.run(W)
    
print(w_evaluated)

x_placeholder = tf.placeholder(dtype=tf.float32, shape=[None, 8], name='x_placeholder')
y_hat = tf.matmul(x_placeholder, W, name='y_hat')

# show_graph(tf.get_default_graph())

with tf.Session() as ses:
    sample_x = [[1, 2, 3, 4, 5, 6, 7, 8]]
    sample_feed_dict = {x_placeholder: sample_x}
    sample_y_hat = ses.run(y_hat, feed_dict=sample_feed_dict)
    
print(sample_y_hat)

with tf.Session() as ses:
    sample_x = [
        [1, 2, 3, 4, 5, 6, 7, 8],
        [1, 1, 1, 1, 2, 2, 2, 2]
               ]
    sample_feed_dict = {x_placeholder: sample_x}
    sample_y_hat = ses.run(y_hat, feed_dict=sample_feed_dict)
    
print(sample_y_hat)

[[ 5.1353359e-01]
 [ 1.5650436e-02]
 [-1.8251571e-01]
 [ 8.6500472e-01]
 [ 7.7881059e-06]
 [-4.6992372e-03]
 [-6.3953139e-02]
 [-1.6385447e-02]]
[[2.8503945]]
[[2.8503945]
 [1.041613 ]]


## Gradient Descent Solution

### One Common Mistake

In [16]:
tf.reset_default_graph()
x = tf.Variable(1, name='x')
y = tf.Variable(2, name='y')
init_operation = tf.global_variables_initializer()

with tf.Session() as ses:
    ses.run(init_operation)
    for i in range(10):
        x = x + y
    print(ses.run(x))

21


In [17]:
for op in tf.get_default_graph().get_operations():
    print(op.name)

x/initial_value
x
x/Assign
x/read
y/initial_value
y
y/Assign
y/read
init
add
add_1
add_2
add_3
add_4
add_5
add_6
add_7
add_8
add_9


### Manually Computing The Gradient

In [18]:
tf.reset_default_graph()
tf.set_random_seed(42)
learning_rate = 0.0000005
epochs = 10000
n_data = len(X_data)
X = tf.constant(X_data, dtype=tf.float32, name='X')
X_transpose = tf.transpose(X, name='X_transpose')
Y = tf.constant(Y_data, dtype=tf.float32, name='Y')
W = tf.Variable(tf.random.normal(shape=[8, 1], mean=0.0, stddev=0.01, name="W"))
Y_hat = tf.matmul(X, W, name="Y_hat")
errors = Y_hat - Y
squared_errors = 0.5 * tf.square(errors, name='squared_errors')
MSE = tf.reduce_mean(squared_errors, name='MSE')
gradients = (1 / n_data) * tf.matmul(X_transpose, errors)
update_W = tf.assign(W, W - learning_rate * gradients)
init_operation = tf.global_variables_initializer()

with tf.Session() as ses:
    ses.run(init_operation)
    for i in range(epochs):
        _, mse = ses.run([update_W, MSE])
        if i % 1000 == 0:
            print("Epoch= ", i, "Mse = ", mse, end="\n")
    
    print()
        
    print(ses.run(W))
    

Epoch=  0 Mse =  32.529778
Epoch=  1000 Mse =  0.6525687
Epoch=  2000 Mse =  0.65019965
Epoch=  3000 Mse =  0.64824283
Epoch=  4000 Mse =  0.6463655
Epoch=  5000 Mse =  0.6445591
Epoch=  6000 Mse =  0.642816
Epoch=  7000 Mse =  0.64112806
Epoch=  8000 Mse =  0.6394914
Epoch=  9000 Mse =  0.63789934

[[ 1.9240571e-02]
 [ 5.9371735e-03]
 [ 2.4913729e-03]
 [-1.1112956e-02]
 [-1.6945979e-06]
 [ 1.0995283e-03]
 [ 1.6693825e-02]
 [-1.0265485e-02]]


### Using Auto Differentiating in Tensorflow 

In [19]:
tf.reset_default_graph()
tf.set_random_seed(42)
learning_rate = 0.0000005
epochs = 10000
n_data = len(X_data)
X = tf.constant(X_data, dtype=tf.float32, name='X')
X_transpose = tf.transpose(X, name='X_transpose')
Y = tf.constant(Y_data, dtype=tf.float32, name='Y')
W = tf.Variable(tf.random.normal(shape=[8, 1], mean=0.0, stddev=0.01, name="W"))
Y_hat = tf.matmul(X, W, name="Y_hat")
errors = Y_hat - Y
squared_errors = 0.5 * tf.square(errors, name='squared_errors')
MSE = tf.reduce_mean(squared_errors, name='MSE')
gradients = tf.gradients(MSE, W)
print(gradients[0])
update_W = tf.assign(W, W - learning_rate * gradients[0])
init_operation = tf.global_variables_initializer()

with tf.Session() as ses:
    ses.run(init_operation)
    for i in range(epochs):
        _, mse = ses.run([update_W, MSE])
        if i % 1000 == 0:
            print("Epoch= ", i, "Mse = ", mse, end="\n")
    
    print()
        
    print(ses.run(W))
    

Tensor("gradients/Y_hat_grad/MatMul_1:0", shape=(8, 1), dtype=float32)
Epoch=  0 Mse =  32.529778
Epoch=  1000 Mse =  0.65256876
Epoch=  2000 Mse =  0.6501996
Epoch=  3000 Mse =  0.64824283
Epoch=  4000 Mse =  0.6463655
Epoch=  5000 Mse =  0.64455914
Epoch=  6000 Mse =  0.642816
Epoch=  7000 Mse =  0.64112806
Epoch=  8000 Mse =  0.6394914
Epoch=  9000 Mse =  0.63789934

[[ 1.92405693e-02]
 [ 5.93717443e-03]
 [ 2.49137124e-03]
 [-1.11129563e-02]
 [-1.69458269e-06]
 [ 1.09952840e-03]
 [ 1.66938305e-02]
 [-1.02654835e-02]]


### Using Tensorflow Optimizers


In [20]:
tf.reset_default_graph()
tf.set_random_seed(42)
learning_rate = 0.0000005
epochs = 10000
n_data = len(X_data)
X = tf.constant(X_data, dtype=tf.float32, name='X')
X_transpose = tf.transpose(X, name='X_transpose')
Y = tf.constant(Y_data, dtype=tf.float32, name='Y')
W = tf.Variable(tf.random.normal(shape=[8, 1], mean=0.0, stddev=0.01, name="W"))
Y_hat = tf.matmul(X, W, name="Y_hat")
errors = Y_hat - Y
squared_errors = 0.5 * tf.square(errors, name='squared_errors')
MSE = tf.reduce_mean(squared_errors, name='MSE')
optimizer = tf.train.GradientDescentOptimizer(learning_rate, name="optimizer")
update_W = optimizer.minimize(MSE)
init_operation = tf.global_variables_initializer()

with tf.Session() as ses:
    ses.run(init_operation)
    for i in range(epochs):
        _, mse = ses.run([update_W, MSE])
        if i % 1000 == 0:
            print("Epoch= ", i, "Mse = ", mse, end="\n")
    
    print()
        
    print(ses.run(W))
    

Epoch=  0 Mse =  32.529778
Epoch=  1000 Mse =  0.65256876
Epoch=  2000 Mse =  0.6501996
Epoch=  3000 Mse =  0.64824283
Epoch=  4000 Mse =  0.6463655
Epoch=  5000 Mse =  0.64455914
Epoch=  6000 Mse =  0.642816
Epoch=  7000 Mse =  0.64112806
Epoch=  8000 Mse =  0.6394914
Epoch=  9000 Mse =  0.63789934

[[ 1.92405693e-02]
 [ 5.93717583e-03]
 [ 2.49137334e-03]
 [-1.11129563e-02]
 [-1.69456109e-06]
 [ 1.09952851e-03]
 [ 1.66938268e-02]
 [-1.02654835e-02]]
